In [1]:
import pandas as pd
import json
from sbscorer.sbdata.FlipsideApi import FlipsideApi
import os
import numpy as np
from pathlib import Path

In [2]:
FOLDER_NAME = 'gr18'
CHAIN = 'optimism'
extract_all = True

# Set path to data folder
current_dir = Path(os.getcwd())
PATH_TO_EXPORT = os.path.join(current_dir.parent.parent, 'tx_data', FOLDER_NAME)

# Initialize Flipside API
api_key = os.environ['FLIPSIDE_API_KEY']
flipside_api = FlipsideApi(api_key, max_address=100)

In [3]:
PATH_TO_EXPORT 

'c:\\Users\\DELL Admin\\Documents\\ODC\\RegenRangers\\tx_data\\gr18'

In [4]:
sql_template = """
 -- forked from gr18 all votes with direct @ https://flipsidecrypto.xyz/edit/queries/a5eb357f-d1b0-4df7-8e35-5848f6dc5209

with 
eth_time AS (
  SELECT
    tx_hash,
    block_timestamp,
    origin_from_address as voter,
    amount_usd,
  origin_to_address,
  eth_to_address
  from
    optimism.core.ez_eth_transfers
  where
    block_timestamp between '2023-08-15 12:00:00.000'
    and '2023-08-29 12:00:00.000'
),
eth_direct as (
  SELECT
    tx_hash,
    block_timestamp,
    voter,
    'ETH' as token,
    amount_usd
  FROM
    eth_time
  WHERE
    origin_to_address IN (
      '0x8de918f0163b2021839a8d84954dd7e8e151326d',
      '0xb6be0ecafdb66dd848b0480db40056ff94a9465d',
      '0x2871742b184633f8dc8546c6301cbc209945033e',
      '0x10be322de44389ded49c0b2b73d8c3a1e3b6d871',
      '0x5b95acf46c73fd116f0fedadcbef453530e35d0',
      '0xc5fdf5cff79e92fac1d6efa725c319248d279200',
      '0xf591e42dffe8e62c2085ccaadfe05f84d89d0c6',
      '0x9331fde4db7b9d9d1498c09d30149929f24cf9d5',
      '0x30c381033aa2830ceb0aa372c2e4d28f004b3db9',
      '0x69e423181f1d3e6bebf8ab88030c36da73785f26'
    )
    and eth_to_address IN (
      '0x8de918f0163b2021839a8d84954dd7e8e151326d',
      '0xb6be0ecafdb66dd848b0480db40056ff94a9465d',
      '0x2871742b184633f8dc8546c6301cbc209945033e',
      '0x10be322de44389ded49c0b2b73d8c3a1e3b6d871',
      '0x5b95acf46c73fd116f0fedadcbef453530e35d0',
      '0xc5fdf5cff79e92fac1d6efa725c319248d279200',
      '0xf591e42dffe8e62c2085ccaadfe05f84d89d0c6',
      '0x9331fde4db7b9d9d1498c09d30149929f24cf9d5',
      '0x30c381033aa2830ceb0aa372c2e4d28f004b3db9',
      '0x69e423181f1d3e6bebf8ab88030c36da73785f26'
    )
),


unique_voter as (SELECT DISTINCT voter 
FROM eth_direct)

SELECT 
tx.TX_HASH,
tx.BLOCK_TIMESTAMP,
tx.FROM_ADDRESS,
tx.TO_ADDRESS,
tx.GAS_LIMIT,
tx.GAS_USED,
tx.TX_FEE,
tx.ETH_VALUE
FROM optimism.core.fact_transactions tx
INNER JOIN
unique_voter v ON tx.FROM_ADDRESS = v.voter OR tx.TO_ADDRESS = v.voter
WHERE 
-- tx.BLOCK_TIMESTAMP 
-- between CURRENT_DATE - interval '%d day'
-- AND CURRENT_DATE - interval '%d day'
-- AND 
tx.STATUS = 'SUCCESS'  
"""

In [5]:
query_result_set = flipside_api.sdk.query(sql_template,
                                              max_age_minutes=flipside_api.MAX_AGE_MINUTES,
                                              page_size=50000,
                                              page_number=1,
                                              timeout_minutes=flipside_api.TIMEOUT_MINUTES,
                                              ttl_minutes=flipside_api.TTL_MINUTES,
                                              cached=flipside_api.CACHED,
                                              retry_interval_seconds=flipside_api.RETRY_INTERVAL_SECONDS)

In [6]:
def human_readable_size(size, decimal_places=2):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024.0:
            return f"{size:.{decimal_places}f} {unit}"
        size /= 1024.0
    return f"{size:.{decimal_places}f} PB"

print(human_readable_size(query_result_set.run_stats.bytes))


23.27 MB


In [9]:
flipside_api.PAGE_SIZE = 25000

In [10]:
df = flipside_api.execute_query(sql_template)

In [12]:
df_loaded = pd.read_csv(os.path.join(PATH_TO_EXPORT, 'gr_18_optimism_all_tx.csv'))

In [13]:
df_loaded.shape

(6776216, 9)

In [14]:
df.shape

(143290, 9)

In [16]:
new_df = pd.concat([df_loaded, df])

In [17]:
'0xa1c3ba69793690a86448990fd34c6a9d4c1f10239725aa135779a5c9ab534067' in new_df['tx_hash'].values

True

In [18]:
df_not_dupplicate = new_df[~new_df.duplicated()]
df_not_dupplicate

,tx_hash,block_timestamp,from_address,to_address,gas_limit,gas_used,tx_fee,eth_value,__row_index
0,0x863c5b543d29b329891fdec341845a73c2525e6dc212...,2023-08-24T20:22:03.000Z,0xf4af0314a06ea7376da5bdecc33f6f9800f1ae28,0x7f5c764cbc14f9669b88837ca1490cca17c31607,53067,53067,0.000029,0.000000,0
1,0xfdffb6eb77e72ca9f54297ebcc7644ea55db521f64c7...,2023-08-24T22:05:41.000Z,0x87dbffd0ed90b2252856faf753437a7bc9fcfa7b,0xb009b5707e6e0dc9830a1262de37f055972e6480,219895,139765,0.000058,0.000777,1
2,0x7b416f625fd29c2da17a7ad35b1c6dc7e0cf5b548249...,2023-08-24T16:10:17.000Z,0x532c1fc2940ec38d5c5e7fc8b8aa4aaddcaa17d3,0x46ce3b08bdef4461fb444d3746fd97f0af11dccf,731592,467423,0.000093,0.000000,2
3,0xbed4ce09f5f5c4d7aac7e310dfce11d94bf86f7da6d5...,2023-08-24T16:09:33.000Z,0x0596c217a79ea00e5dafc43778138ea61fd67ee2,0x2e42f214467f647fe687fd9a2bf3baddfa737465,119625,119625,0.000087,0.000000,3
4,0xb60502458c8de31606b84e0aed102ac83629726d8734...,2023-08-24T22:08:15.000Z,0xc1991c8bda29507991eeb230ff2063c2ea74a34c,0xa062ae8a9c5e11aaa026fc2670b0d65ccc8b2858,246899,229592,0.000070,0.000000,4
...,...,...,...,...,...,...,...,...,...
18285,0x95bca0fe01d9cb3965c87e38e933a615c246886c4bf9...,2022-12-03T16:18:40.000Z,0x4e16c6d4ab2648b18f921ecf5ee2a8c2eba78f31,0x69dd38645f7457be13571a847ffd905f9acbaf6d,201477,176260,0.000097,0.000000,118285
18286,0x6f29783f12937dbc183388189c68ffd7fdab3b0ca757...,2022-12-03T09:32:42.000Z,0x7117248ddc900ad01b78c02462b09f3c594b13ea,0x7f5c764cbc14f9669b88837ca1490cca17c31607,53055,53055,0.000048,0.000000,118286
18287,0x40153174c0f084b67269afbb40f187e85949adcb44d2...,2022-12-03T19:23:40.000Z,0x4b040a88cb4edd02b97f0ab8c97d3da540fc2cf0,0x7f5c764cbc14f9669b88837ca1490cca17c31607,70000,53079,0.000060,0.000000,118287
18288,0x1bb8b885544c5098b0339131dcdc442fdd98b5956c74...,2022-12-03T12:45:47.000Z,0x4165ac75516763dae3481a5143f31f7b11ed618b,0x2e42f214467f647fe687fd9a2bf3baddfa737465,119630,119630,0.000069,0.000000,118288


In [21]:
df_not_dupplicate.shape[0] - new_df.shape[0]

-25000

In [22]:
if not os.path.exists(PATH_TO_EXPORT):
    os.makedirs(PATH_TO_EXPORT)

df_not_dupplicate.to_csv(os.path.join(PATH_TO_EXPORT, 'gr_18_optimism_all_tx.csv'), index=False)
df_not_dupplicate.to_pickle(os.path.join(PATH_TO_EXPORT, 'gr_18_optimism_all_tx.pkl'))

In [23]:
PATH_TO_EXPORT

'c:\\Users\\DELL Admin\\Documents\\ODC\\RegenRangers\\tx_data\\gr18'